In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd




In [2]:
train = pd.read_csv('/Users/xianglongtan/Desktop/cmps242/week6/train_clean.csv',encoding='ISO-8859-1')
test = pd.read_csv('/Users/xianglongtan/Desktop/cmps242/week6/test_clean.csv',encoding='ISO-8859-1')

# Preprocessing Training Set

In [3]:
# tokenization
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
filter_tweet = [tweet for tweet in train.tweet.values]
word_token = [word_tokenize(tweet) for tweet in filter_tweet]
filter_words = []
for word in word_token:
    filter_words.append([w for w in word if not w in stop_words])
filter_tweet = []
for word in filter_words:
    filter_tweet.append(' '.join(word))

In [4]:
# process label
train[['D','H']] = pd.get_dummies(train['handle'])

In [5]:
train.head()

,handle,tweet,D,H
0,realDonaldTrump,"In Tampa, Florida- thank you to all of our out...",0,1
1,realDonaldTrump,Poll: <mention> realDonaldTrump vs. <mention> ...,0,1
2,HillaryClinton,Obama on whether Trump could be trusted with U...,1,0
3,HillaryClinton,"""Hillary Clinton has never quit on anything in...",1,0
4,realDonaldTrump,I LOVE NEW YORK! <hashtag> NewYorkValues <url>,0,1


# tfidf transform

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
vectorizer = CountVectorizer()
train_token = vectorizer.fit_transform(filter_tweet)
transformer = TfidfTransformer(smooth_idf = True)
tfidf_train_token = transformer.fit_transform(train_token)
tfidf_train_token = tfidf_train_token.toarray()
colname = vectorizer.get_feature_names()
df_token = pd.DataFrame(data=tfidf_train_token,columns=colname)



In [7]:
df_token.shape

(5000, 8410)

# PCA

In [8]:
# Standardizer
from sklearn import preprocessing
standardScaler = preprocessing.StandardScaler(with_mean=False)
standard_df_token = standardScaler.fit_transform(df_token)

In [9]:
# PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=2000)
df_token_pca=pca.fit_transform(standard_df_token)
pca.explained_variance_ratio_.sum()

0.7597408503755808

In [10]:
standard_test_token = standardScaler.transform(df_token_test)
test_token_pca = pca.transform(standard_test_token)
pca.explained_variance_ratio_.sum()

NameError: name 'df_token_test' is not defined

# Train with SVM

In [8]:
#X = df_token_pca
X = df_token
Y = train.D

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train,X_test,y_train,y_test = train_test_split(X,Y)
sigmoid_svc = svm.SVC(kernel = 'sigmoid')
sigmoid_svc.fit(X_train,y_train)
y_pred = sigmoid_svc.predict(X_test)
prediction = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test,prediction)



/Users/xianglongtan/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:194: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


# cross validation

In [14]:
param_grid = [{'kernel':['rbf'], 'gamma':[1e-3,1e-2,1e-1],
        'C':[1, 10, 100, 1000]},
        {'kernel':['linear'] , 'C':[1,10,100,1000]},
        {'kernel':['poly'], 'gamma':[1e-3,1e-2,1e-1],'C':[1, 10, 100, 1000]},
        {'kernel':['sigmoid'], 'gamma':[1e-3,1e-2,1e-1],'C':[1, 10, 100, 1000]}]

In [18]:
seed=4
import time
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
model = svm.SVC()
kfold = StratifiedKFold(n_splits = 10, shuffle=True,random_state = seed)
grid_search = GridSearchCV(model, param_grid, scoring = 'accuracy', cv=kfold)
start = time.time()
grid_result = grid_search.fit(X,Y)
end = time.time()
print(end-start)

KeyboardInterrupt: 

In [ ]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean,stdev,param in zip(means, stds, params):
    print('%f (%f) with: %r'%(mean,stdev,param))

In [ ]:
print('best score: ',grid_search.best_score_,'\nbest param:',grid_result.best_params_)

# predict testset

In [11]:
test_ = [tweet for tweet in test.tweet.values]
test_ = [word_tokenize(tweet) for tweet in test_]
filter_test = []
for word in test_:
    filter_test.append([w for w in word if not w in stop_words])
filter_tweet_test = []
for word in filter_test:
    filter_tweet_test.append(' '.join(word))
test_token = vectorizer.transform(filter_tweet_test)
tfidf_test_token = transformer.transform(test_token)
tfidf_test_token = tfidf_test_token.toarray()
colname = vectorizer.get_feature_names()
df_token_test = pd.DataFrame(data=tfidf_test_token,columns=colname)


In [12]:
# pca
standard_test_token = standardScaler.transform(df_token_test)
test_token_pca = pca.transform(standard_test_token)
pca.explained_variance_ratio_.sum()

0.7597408503755808

In [22]:
X_test = test_token_pca# pca
#X_test = df_token_test# non pca
X_test.shape
#X = df_token# non pca
X = df_token_pca# pca
Y = train.H
X.shape

(5000, 2000)

In [29]:
from sklearn import svm
model = svm.SVC(kernel = 'linear',C = 10, probability = True)
model.fit(X,Y)
pred = model.predict(X)
accuracy_score(Y,pred)

0.9996

In [30]:
result = model.predict(X_test)
test_pred = pd.DataFrame(data=result,columns=['realDonaldTrump'])
test_pred['HillaryClinton'] = 1- test_pred['realDonaldTrump']
test_pred.index.name = 'id'
test_pred.to_csv('HW5_SVM.csv')

In [31]:
train['pred_H'] = pred
train.head(20)

,handle,tweet,D,H,pred_H,ans_train
0,realDonaldTrump,"In Tampa, Florida- thank you to all of our out...",0,1,1,0.441013
1,realDonaldTrump,Poll: <mention> realDonaldTrump vs. <mention> ...,0,1,1,0.368204
2,HillaryClinton,Obama on whether Trump could be trusted with U...,1,0,0,0.625429
3,HillaryClinton,"""Hillary Clinton has never quit on anything in...",1,0,0,0.677193
4,realDonaldTrump,I LOVE NEW YORK! <hashtag> NewYorkValues <url>,0,1,1,0.179583
5,HillaryClinton,"""In times like these, we need a president who ...",1,0,0,0.664918
6,realDonaldTrump,Doesn't fit the MSM narrative - so they wont s...,0,1,1,0.441032
7,HillaryClinton,BREAKING: Clinton is projected winner in U.S. ...,1,0,0,0.550093
8,HillaryClinton,"This fall, USPS will honor joyous Hindu Festiv...",1,0,0,0.550107
9,realDonaldTrump,HILLARY'S BAD TAX HABIT! <url>,0,1,1,0.441010


In [44]:
result2 = model.predict_proba(X_test)


In [47]:
test_pred2 = pd.DataFrame(data=result2,columns=['HillaryClinton','realDonaldTrump'])
#test_pred2['HillaryClinton'] = 1- test_pred2['realDonaldTrump']
test_pred2.index.name = 'id'
test_pred2.to_csv('HW5_SVM3.csv')
test_pred2.head()

,HillaryClinton,realDonaldTrump
id,,
0,0.690588,0.309412
1,0.538222,0.461778
2,0.210770,0.789230
3,0.598999,0.401001
4,0.868278,0.131722


In [39]:
ans_train = model.predict_proba(X)
#train[['ansD','ansH']] = ans_train
#train.head()


array([[0.45054772, 0.54945228],
       [0.36139536, 0.63860464],
       [0.60683901, 0.39316099],
       ...,
       [0.6293079 , 0.3706921 ],
       [0.80662124, 0.19337876],
       [0.58796891, 0.41203109]])

In [43]:
print(ans_train[0:5,:])
train.head()

[[0.45054772 0.54945228]
 [0.36139536 0.63860464]
 [0.60683901 0.39316099]
 [0.70457455 0.29542545]
 [0.1947527  0.8052473 ]]


,handle,tweet,D,H,pred_H,ans_train
0,realDonaldTrump,"In Tampa, Florida- thank you to all of our out...",0,1,1,0.450548
1,realDonaldTrump,Poll: <mention> realDonaldTrump vs. <mention> ...,0,1,1,0.361395
2,HillaryClinton,Obama on whether Trump could be trusted with U...,1,0,0,0.606839
3,HillaryClinton,"""Hillary Clinton has never quit on anything in...",1,0,0,0.704575
4,realDonaldTrump,I LOVE NEW YORK! <hashtag> NewYorkValues <url>,0,1,1,0.194753
